# Latent Diffusion Models

In [ ]:
!rm -rf taming-transformers
!git clone https://github.com/CompVis/taming-transformers.git
!{sys.executable} -m pip install -e taming-transformers
!find /content/taming-transformers -type d -name taming
!rm -rf Different-Projects
!git clone --no-checkout https://github.com/edlaf/Different-Projects.git
%cd Different-Projects
!git sparse-checkout init --no-cone
!echo "/*" > .git/info/sparse-checkout
!echo "!/Deep Learning/Datasets" >> .git/info/sparse-checkout
!git checkout
%cd Different-Projects
!git pull

In [ ]:
import sys
sys.path.append('/content/Different-Projects/Deep Learning/Models/Latent Diffusion Model ')
sys.path.append('/content/taming-transformers')
import AutoEncoder as AutoEncoder
import AutoEncoder_models as Autoencoder_models
import models_utils as utils
import Diffusion_Model as diff_model
import torch
import torch.nn as nn
import kagglehub
path = kagglehub.dataset_download("badasstechie/celebahq-resized-256x256")
from google.colab import drive
drive.mount('/content/drive')

## Auto-Encoder

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
train, test, data_ = AutoEncoder.load_data(path, batch_size=64)

images, _ = next(iter(train))

AutoEncoder.show(images)

In [ ]:
chanels = 4
autoencoder_model = Autoencoder_models.Auto_Encoder(latent_channels=chanels).to(device)
criterion = "loss perso aller dans model.loss (fichier Autoencoder_model)"
optimizer = torch.optim.Adam(autoencoder_model.parameters(), lr=0.001)
nb_params = sum([p.numel() for p in autoencoder_model.parameters()])
print(f"Number of parameters {nb_params}")

In [ ]:
AutoEncoder.train(autoencoder_model, train, optimizer, test,  device, nb_epochs = 70, display_interval = 1)

In [ ]:
torch.save(autoencoder_model.state_dict(), "/content/drive/MyDrive/autoencoder_weights.pt")
print("Saved")

state_dict = torch.load("/content/drive/MyDrive/autoencoder_weights.pt", map_location=torch.device(device))
autoencoder_model.load_state_dict(state_dict)

In [ ]:
AutoEncoder.test(autoencoder_model, criterion, test, device)

In [ ]:
encoded_data_ = AutoEncoder.Encode_data(autoencoder_model, device, data_)

## Diffusion Model

In [ ]:
batch_size = 128
model = utils.UNet(in_ch=chanels)
optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)
diffusion_model = diff_model.DiffusionModel(1000, model, device)
nb_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Number of parameters: {nb_params}")

In [ ]:
with torch.no_grad():
    generated_images = diffusion_model.sampling(n_samples=25, image_channels=chanels, img_size=(32, 32))
    decoded_generated_images = autoencoder_model.decoder(generated_images)
    AutoEncoder.show(decoded_generated_images)

In [ ]:
diffusion_model.training(encoded_data_, optimizer, autoencoder_model, nb_epochs=150)

In [ ]:
torch.save(diffusion_model.state_dict(), "/content/drive/MyDrive/diffusion_model_weights.pt")
print("Saved")

state_dict = torch.load("/content/drive/MyDrive/diffusion_model_weights.pt", map_location=torch.device(device))
diffusion_model.load_state_dict(state_dict)
diffusion_model.eval()

In [ ]:
with torch.no_grad():
    generated_images = diffusion_model.sampling(n_samples=25, image_channels=chanels, img_size=(32, 32))
    decoded_generated_images = autoencoder_model.decoder(generated_images)
    AutoEncoder.show(decoded_generated_images)